In [1]:
cd set2

/media/datastorage/Phong/ufc101/set2


In [2]:
ls -l

total 32503460
-rw-rw-r-- 1 bribeiro bribeiro        107 jan 30 23:23 checkpoint
-rw-rw-r-- 1 bribeiro bribeiro  138864569 jan 22 00:50 DenseNet201_CNNRNN2-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 22 00:50 DenseNet201_CNNRNN2-set2.index
-rw-rw-r-- 1 bribeiro bribeiro 2868085648 jan 19 21:06 DenseNet201_CNNRNN_test_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      30000 jan 19 21:08 DenseNet201_CNNRNN_test_labels_set2.npy
-rw-rw-r-- 1 bribeiro bribeiro 7363006852 jan 19 21:05 DenseNet201_CNNRNN_train_100_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      76816 jan 19 21:08 DenseNet201_CNNRNN_train_labels_set2.npy
-rw-rw-r-- 1 bribeiro bribeiro 1067756108 jan 26 03:25 DenseNet201_Transformer-set2.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro       4586 jan 26 03:25 DenseNet201_Transformer-set2.index
-rw-rw-r-- 1 bribeiro bribeiro 1070762088 jan 30 23:07 EffB4_CNNRNN_test_40_set2.pickle
-rw-rw-r-- 1 bribeiro bribeiro      30000 jan 30 23:07 EffB4_CNNRNN_t

In [3]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [10]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

MAX_SEQ_LENGTH = 40
NUM_FEATURES = 1024

In [11]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 9586
Total videos for testing: 3734


,video_name,tag
4847,v_LongJump_g01_c06.avi,LongJump
8111,v_SoccerPenalty_g23_c01.avi,SoccerPenalty
103,v_ApplyEyeMakeup_g25_c05.avi,ApplyEyeMakeup
8647,v_TaiChi_g23_c04.avi,TaiChi
8844,v_ThrowDiscus_g21_c04.avi,ThrowDiscus
7873,v_SkyDiving_g15_c02.avi,SkyDiving
2342,v_CricketShot_g25_c02.avi,CricketShot
1884,v_BrushingTeeth_g06_c03.avi,BrushingTeeth
5819,v_PlayingDhol_g15_c03.avi,PlayingDhol
5944,v_PlayingFlute_g16_c06.avi,PlayingFlute


In [12]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [13]:

def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input 

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [14]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

In [15]:
from tqdm import tqdm

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm(enumerate(video_paths)):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


# train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

# print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")

3734it [2:39:15,  2.56s/it]


In [16]:
train_data, train_labels = prepare_all_videos(train_df, "train")

9586it [6:45:43,  2.54s/it]


In [ ]:
# # Utility for our sequence model.
# # Bidirectional

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5))(#16, 512
#         frame_features_input, mask=mask_input
#     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.LeakyReLU()(x)
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
#     return rnn_model


# # Utility for running experiments.
# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     history = seq_model.fit(
#         [train_data[0], train_data[1]],
#         train_labels,
#         validation_split=0.2,
#         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# _, sequence_model = run_experiment()

In [ ]:
#40.13% LSTM(2048,dropout=0.1)
#46.47% LSTM(2048,dropout=0.5)
#43.96% LSTM(4096,dropout=0.5)

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=True,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# # x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:

# def prepare_single_video(frames):
#     frames = frames[None, ...]
#     frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#     frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

#     for i, batch in enumerate(frames):
#         video_length = batch.shape[0]
#         length = min(MAX_SEQ_LENGTH, video_length)
#         for j in range(length):
#             frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
#         frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#     return frame_features, frame_mask


# def sequence_prediction(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features, frame_mask = prepare_single_video(frames)
#     probabilities = sequence_model.predict([frame_features, frame_mask])[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     return frames


# # This utility is for visualization.
# # Referenced from:
# # https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


# test_video = np.random.choice(test_df["video_name"].values.tolist())
# print(f"Test video path: {test_video}")
# test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

In [ ]:
ls -l

In [ ]:
# import numpy as np
# import tensorflow as tf 

# np.save('Inception_CNNRNN_train_data.npy', train_data)    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_test_data.npy', test_data[0])    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_train_labels.npy', train_data[1])    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
# print(train_data[0].shape)
# print(train_data[1].shape)

In [ ]:
# print(train_data.type)

In [17]:
import pickle

with open('DenseNet121_CNNRNN_train_40_set2.pickle', 'wb') as f:
    pickle.dump(train_data, f)

In [21]:
import pickle

with open('DenseNet121_CNNRNN_test_40_set2.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [19]:
import numpy as np
import tensorflow as tf 

# np.save('densenet_train_data.npy', train_data)    # .npy extension is added if not given
# np.save('densenet_test_data.npy', test_data)    # .npy extension is added if not given
np.save('DenseNet121_CNNRNN_train_40_labels_set2.npy', train_labels)    # .npy extension is added if not given
np.save('DenseNet121_CNNRNN_test_40_labels_set2.npy', test_labels)    # .npy extension is added if not given

In [ ]:
# ls -l

In [ ]:
# mv Inception_CNNRNN_20.pickle Inception_CNNRNN_train_20.pickle

In [23]:
import numpy as np
import tensorflow as tf 
import pickle

with open('DenseNet121_CNNRNN_train_40_set2.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('DenseNet121_CNNRNN_test_40_set2.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('DenseNet121_CNNRNN_train_40_labels_set2.npy')
d_test_labels = np.load('DenseNet121_CNNRNN_test_40_labels_set2.npy')

In [26]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.4)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "DenseNet121_CNNRNN2-set2-seq40"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    optimizer = keras.optimizers.SGD(lr=1e-1)
#     optimizer = keras.optimizers.Adadelta()
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
    
    history = seq_model.fit(
        [d_train_data[0], d_train_data[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data[0], d_test_data[1]],
        d_test_labels),
        epochs=100,
#         epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


d_run_experiment()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 40, 1024)]   0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 40)]         0           []                               
                                                                                                  
 lstm_2 (LSTM)                  (None, 2048)         25174016    ['input_9[0][0]',                
                                                                  'input_10[0][0]']               
                                                                                                  
 dense_4 (Dense)                (None, 1024)         2098176     ['lstm_2[0][0]']           

2023-01-31 14:12:30.846737: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1570570240 exceeds 10% of free system memory.


Epoch 1/100
299/300 [============================>.] - ETA: 0s - loss: 2.1449 - accuracy: 0.4540
Epoch 1: val_loss improved from inf to 1.94693, saving model to DenseNet121_CNNRNN2-set2-seq40
300/300 [==============================] - 20s 57ms/step - loss: 2.1444 - accuracy: 0.4542 - val_loss: 1.9469 - val_accuracy: 0.4968
Epoch 2/100
299/300 [============================>.] - ETA: 0s - loss: 0.8916 - accuracy: 0.7334
Epoch 2: val_loss improved from 1.94693 to 1.40356, saving model to DenseNet121_CNNRNN2-set2-seq40
300/300 [==============================] - 15s 51ms/step - loss: 0.8909 - accuracy: 0.7336 - val_loss: 1.4036 - val_accuracy: 0.6299
Epoch 3/100
299/300 [============================>.] - ETA: 0s - loss: 0.5998 - accuracy: 0.8175
Epoch 3: val_loss improved from 1.40356 to 1.37563, saving model to DenseNet121_CNNRNN2-set2-seq40
300/300 [==============================] - 16s 52ms/step - loss: 0.5991 - accuracy: 0.8178 - val_loss: 1.3756 - val_accuracy: 0.6583
Epoch 4/100
299/3

299/300 [============================>.] - ETA: 0s - loss: 0.0052 - accuracy: 0.9990
Epoch 29: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 52ms/step - loss: 0.0052 - accuracy: 0.9990 - val_loss: 1.5209 - val_accuracy: 0.7493
Epoch 30/100
299/300 [============================>.] - ETA: 0s - loss: 0.0031 - accuracy: 0.9994
Epoch 30: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 52ms/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 1.5183 - val_accuracy: 0.7464
Epoch 31/100
300/300 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 0.9994
Epoch 31: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 52ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 1.5622 - val_accuracy: 0.7450
Epoch 32/100
299/300 [============================>.] - ETA: 0s - loss: 9.7158e-04 - accuracy: 0.9999
Epoch 32: val_loss did not improve from 1.24680
300/300 [=================

Epoch 58/100
299/300 [============================>.] - ETA: 0s - loss: 1.2325e-04 - accuracy: 1.0000
Epoch 58: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 53ms/step - loss: 1.2307e-04 - accuracy: 1.0000 - val_loss: 1.6454 - val_accuracy: 0.7523
Epoch 59/100
299/300 [============================>.] - ETA: 0s - loss: 1.1805e-04 - accuracy: 1.0000
Epoch 59: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 53ms/step - loss: 1.1800e-04 - accuracy: 1.0000 - val_loss: 1.6422 - val_accuracy: 0.7528
Epoch 60/100
299/300 [============================>.] - ETA: 0s - loss: 1.6611e-04 - accuracy: 1.0000
Epoch 60: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 52ms/step - loss: 1.6593e-04 - accuracy: 1.0000 - val_loss: 1.6436 - val_accuracy: 0.7523
Epoch 61/100
299/300 [============================>.] - ETA: 0s - loss: 1.1099e-04 - accuracy: 1.0000
Epoch 61: val_loss did not improve fr

Epoch 87/100
299/300 [============================>.] - ETA: 0s - loss: 7.8783e-05 - accuracy: 1.0000
Epoch 87: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 53ms/step - loss: 7.8767e-05 - accuracy: 1.0000 - val_loss: 1.6710 - val_accuracy: 0.7472
Epoch 88/100
299/300 [============================>.] - ETA: 0s - loss: 1.2919e-04 - accuracy: 1.0000
Epoch 88: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 52ms/step - loss: 1.2899e-04 - accuracy: 1.0000 - val_loss: 1.6738 - val_accuracy: 0.7512
Epoch 89/100
299/300 [============================>.] - ETA: 0s - loss: 9.9075e-05 - accuracy: 1.0000
Epoch 89: val_loss did not improve from 1.24680
300/300 [==============================] - 16s 54ms/step - loss: 9.8916e-05 - accuracy: 1.0000 - val_loss: 1.6721 - val_accuracy: 0.7520
Epoch 90/100
299/300 [============================>.] - ETA: 0s - loss: 1.1789e-04 - accuracy: 1.0000
Epoch 90: val_loss did not improve fr

(<keras.callbacks.History at 0x7fc87ec8abe0>,
 <keras.engine.functional.Functional at 0x7fc8744cdb50>)

In [ ]:
#lr=0.1 = 0.7560
#lr=0.01 = 0.7732
#lr=0.001 = 0.7670

In [ ]:
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    

#     lr_schedule = keras.callbacks.LearningRateScheduler(
#                   lambda epoch: 1e-6 * 10**(4*epoch / 10))
    
#     optimizer = keras.optimizers.SGD(lr=1e-3)
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
# #     seq_model.compile(optimizer=optimizer,
# #                       loss='categorical_crossentropy',
# #                      metrics=['accuracy'])     
    
#     history = seq_model.fit(
#         [d_train_data[0], d_train_data[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data[0], d_test_data[1]],
#         d_test_labels),
#         epochs=20,
# #         epochs=EPOCHS,
#         callbacks=[lr_schedule,checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# his, seq = d_run_experiment()


In [ ]:
# from matplotlib import pyplot as plt

# plt.semilogx(his.history['lr'], his.history['accuracy'])
# plt.axis([1e-6, 1, 0, 1])
# plt.xlabel('lr')
# plt.ylabel('accuracy')
# plt.show()

In [ ]:
# print(d_train_data[0].shape)
# print(d_train_data[1].shape)

In [ ]:
# print(d_train_data[1][0][39])

In [10]:
import numpy as np

# set length of video sequence and indices
MAX_SEQ_LENGTH = 50#override max sequence length
#max_seq=40
# copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# copy_indices=[0,4,8,12,16,20,24,28,32,36]
# copy_indices=[0,5,10,15,20,25,30,35]
# copy_indices=[0,8,16,24,32]
# copy_indices=[0,19]
# copy_indices=[0]
# max_seq=100
# # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
#               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
#               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
#               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
copy_indices=[0,2,4,6,8,10,12,14,16,18,
              20,22,24,26,28,30,32,34,36,38,
              40,42,44,46,48,50,52,54,56,58,
              60,62,64,66,68,70,72,74,76,78,
              80,82,84,86,88,90,92,94,96,98]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# copy_indices=[0,10,20,30,40,50,60,70,80,90]
# copy_indices=[0,20,40,60,80]
# copy_indices=[0,25,50,75]
# copy_indices=[0,50]
d_train_data2_1 = np.copy(d_train_data[0][:,copy_indices, ])
d_train_data2_2 = np.copy(d_train_data[1][:,copy_indices, ])
d_train_data2 = (d_train_data2_1,d_train_data2_2)

d_test_data2_1 = np.copy(d_test_data[0][:,copy_indices, ])
d_test_data2_2 = np.copy(d_test_data[1][:,copy_indices, ])
d_test_data2 = (d_test_data2_1,d_test_data2_2)
# print(d_train_data2.shape)

label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.4)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "DenseNet201_CNNRNN2-set2"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    optimizer = keras.optimizers.SGD(lr=1e-2)
#     optimizer = keras.optimizers.Adadelta()
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
    
    history = seq_model.fit(
        [d_train_data2[0], d_train_data2[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data2[0], d_test_data2[1]],
        d_test_labels),
        epochs=100,
#         epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data2[0], d_test_data2[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


d_run_experiment()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 50, 1920)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 2048)         32514048    ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1024)         2098176     ['lstm[0][0]']               

/home/bribeiro/miniconda3/envs/ar/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2023-01-22 00:47:31.329971: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3681024000 exceeds 10% of free system memory.
2023-01-22 00:47:33.792844: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3681024000 exceeds 10% of free system memory.


Epoch 1/100


2023-01-22 00:47:38.692879: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


300/300 [==============================] - ETA: 0s - loss: 3.5235 - accuracy: 0.2887
Epoch 1: val_loss improved from inf to 1.86141, saving model to DenseNet201_CNNRNN2-set2
300/300 [==============================] - 28s 80ms/step - loss: 3.5235 - accuracy: 0.2887 - val_loss: 1.8614 - val_accuracy: 0.5924
Epoch 2/100
300/300 [==============================] - ETA: 0s - loss: 1.0692 - accuracy: 0.7753
Epoch 2: val_loss improved from 1.86141 to 1.06824, saving model to DenseNet201_CNNRNN2-set2
300/300 [==============================] - 21s 71ms/step - loss: 1.0692 - accuracy: 0.7753 - val_loss: 1.0682 - val_accuracy: 0.7100
Epoch 3/100
300/300 [==============================] - ETA: 0s - loss: 0.3961 - accuracy: 0.9129
Epoch 3: val_loss improved from 1.06824 to 0.91357, saving model to DenseNet201_CNNRNN2-set2
300/300 [==============================] - 21s 71ms/step - loss: 0.3961 - accuracy: 0.9129 - val_loss: 0.9136 - val_accuracy: 0.7405
Epoch 4/100
300/300 [==========================

300/300 [==============================] - ETA: 0s - loss: 7.4980e-04 - accuracy: 1.0000
Epoch 58: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 70ms/step - loss: 7.4980e-04 - accuracy: 1.0000 - val_loss: 0.8784 - val_accuracy: 0.7973
Epoch 59/100
300/300 [==============================] - ETA: 0s - loss: 7.2513e-04 - accuracy: 1.0000
Epoch 59: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 70ms/step - loss: 7.2513e-04 - accuracy: 1.0000 - val_loss: 0.8852 - val_accuracy: 0.7978
Epoch 60/100
300/300 [==============================] - ETA: 0s - loss: 7.2095e-04 - accuracy: 1.0000
Epoch 60: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 70ms/step - loss: 7.2095e-04 - accuracy: 1.0000 - val_loss: 0.8851 - val_accuracy: 0.7981
Epoch 61/100
300/300 [==============================] - ETA: 0s - loss: 6.9554e-04 - accuracy: 1.0000
Epoch 61: val_loss did not improve from 0.80296
30

Epoch 87/100
300/300 [==============================] - ETA: 0s - loss: 4.7854e-04 - accuracy: 1.0000
Epoch 87: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 71ms/step - loss: 4.7854e-04 - accuracy: 1.0000 - val_loss: 0.9011 - val_accuracy: 0.7983
Epoch 88/100
300/300 [==============================] - ETA: 0s - loss: 4.5107e-04 - accuracy: 1.0000
Epoch 88: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 71ms/step - loss: 4.5107e-04 - accuracy: 1.0000 - val_loss: 0.9018 - val_accuracy: 0.7975
Epoch 89/100
300/300 [==============================] - ETA: 0s - loss: 4.4424e-04 - accuracy: 1.0000
Epoch 89: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 71ms/step - loss: 4.4424e-04 - accuracy: 1.0000 - val_loss: 0.9009 - val_accuracy: 0.7973
Epoch 90/100
300/300 [==============================] - ETA: 0s - loss: 4.4235e-04 - accuracy: 1.0000
Epoch 90: val_loss did not improve fr

(<keras.callbacks.History at 0x7f4b7ef3cfd0>,
 <keras.engine.functional.Functional at 0x7f4b7efde220>)

In [ ]:
#max seq = 40
#seq 1 = 75.33
#seq 2 = 78.52
#seq 5 = 79.38
#seq 8 = 79.38
#seq 10 = 79.78
#seq 20 = 78.68

#max seq = 100
#seq 2 = 78.79
#seq 4 = 80.34
#seq 5 = 80.88
#seq 10 = 80.40
#seq 20 = 81.12
#seq 50 = 80.02